In [1]:
import os
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base

# LCEL(LangChain Expression Language)
- 一个最简单示例
<hr>

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("给我讲一个关于 {topic}的笑话")
model = ChatOpenAI(model="gpt-4")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "冰激凌"})

'在一家冰激凌店里，一个小男孩问店员：“你们有番茄味的冰激凌吗？”\n店员答：“抱歉，我们没有。”\n小男孩又问：“那有大葱味的吗？”\n店员困惑地答：“没有，我们只有甜的口味。”\n小男孩再问：“那有鱼露味的吗？”\n店员无奈地答：“我说过了，我们只有甜的口味。”\n小男孩满足地点头，说：“好的，那给我两球草莓味的。”\n店员：“为什么你要问那么多奇怪的口味？”\n小男孩笑着说：“我只是想确认，你们是不是真的只有甜的口味。”'

Prompt

In [3]:
prompt_value = prompt.invoke({"topic": "刺猬"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='给我讲一个关于 刺猬的笑话')])

In [4]:
prompt_value.to_messages()

[HumanMessage(content='给我讲一个关于 刺猬的笑话')]

In [5]:
prompt_value.to_string()

'Human: 给我讲一个关于 刺猬的笑话'

model

In [6]:
message = model.invoke(prompt_value)
message

AIMessage(content='一只兔子和一只刺猬赛跑，刺猬以微弱的优势赢了。\n\n兔子很不服气，说：“你只是运气好，我一定能赢你。”\n\n刺猬笑了笑，说：“好啊，那我们再比一次。”\n\n兔子问：“你为什么笑？”\n\n刺猬回答：“因为我知道，无论我跑多慢，你都会在我背后。” \n\n兔子：“为什么？”\n\n刺猬：“因为你不敢超过我，怕被我刺到。”', response_metadata={'finish_reason': 'stop', 'logprobs': None})

使用llm的区别

In [7]:
from langchain_openai.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm.invoke(prompt_value)

'\n\nRobot: 为什么刺猬的生日派对总是很失败？因为他们总是把蜡烛都弄灭了！'

Output parser

In [8]:
output_parser.invoke(message)

'一只兔子和一只刺猬赛跑，刺猬以微弱的优势赢了。\n\n兔子很不服气，说：“你只是运气好，我一定能赢你。”\n\n刺猬笑了笑，说：“好啊，那我们再比一次。”\n\n兔子问：“你为什么笑？”\n\n刺猬回答：“因为我知道，无论我跑多慢，你都会在我背后。” \n\n兔子：“为什么？”\n\n刺猬：“因为你不敢超过我，怕被我刺到。”'

LCEL的Pipeline：
- ![Alt Text](lcel01.png)

# RAG Search Exampl
- 建立向量数据
- 使用RAG增强
<hr>

In [9]:
! pip install --upgrade --quiet  langchain langchain-openai faiss-cpu tiktoken

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tavily-python 0.3.0 requires tiktoken==0.5.1, but you have tiktoken 0.6.0 which is incompatible.
doctran 0.0.14 requires openai<0.28.0,>=0.27.8, but you have openai 1.14.0 which is incompatible.
doctran 0.0.14 requires tiktoken<0.6.0,>=0.5.0, but you have tiktoken 0.6.0 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [10]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [11]:
chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

自定义也非常简单

In [12]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [13]:
chain.invoke({"question": "where did harrison work", "language": "chinese"})

'Harrison 在 Kensho 工作。'